## Deliverable 2. Create a Customer Travel Destinations Map.

In [30]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import gmaps
#Import The Google API key.
from config import weather_api_key, g_key
# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [31]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,80.89,81,91,21.90,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.47,74,100,13.20,overcast clouds
2,2,Tiksi,RU,71.6872,128.8694,-25.46,85,100,7.23,overcast clouds
3,3,Briancon,FR,44.9000,6.6500,45.00,93,100,7.81,moderate rain
4,4,Punta Arenas,CL,-53.1500,-70.9167,51.80,50,40,18.41,scattered clouds


In [32]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [33]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
clean_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,80.89,81,91,21.90,overcast clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.47,74,100,13.20,overcast clouds
5,5,Sao Joao Da Barra,BR,-21.6403,-41.0511,80.60,74,75,11.50,broken clouds
11,11,Mahebourg,MU,-20.4081,57.7000,80.60,74,20,8.05,few clouds
13,13,Sinnamary,GF,5.3833,-52.9500,80.17,84,92,14.94,overcast clouds
15,15,Taoudenni,ML,22.6783,-3.9836,80.13,19,54,17.34,broken clouds
17,17,Hermanus,ZA,-34.4187,19.2345,73.00,77,1,5.01,clear sky
18,18,Saint-Philippe,RE,-21.3585,55.7679,82.40,74,40,8.05,light rain
19,19,Bambous Virieux,MU,-20.3428,57.7575,80.60,74,20,8.05,few clouds
23,23,Jawhar,SO,2.7809,45.5005,76.41,65,0,17.63,clear sky


In [34]:
# 4a. Determine if there are any empty rows.
clean_df.isnull().any()

City_ID                False
City                   False
Country                 True
Lat                    False
Lng                    False
Max Temp               False
Humidity               False
Cloudiness             False
Wind Speed             False
Current Description    False
dtype: bool

In [59]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_hotel_df = clean_df.dropna()

In [61]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_hotel_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,80.89,overcast clouds,3.0707,172.7902,
1,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,
5,Sao Joao Da Barra,BR,80.60,broken clouds,-21.6403,-41.0511,
11,Mahebourg,MU,80.60,few clouds,-20.4081,57.7000,
13,Sinnamary,GF,80.17,overcast clouds,5.3833,-52.9500,
15,Taoudenni,ML,80.13,broken clouds,22.6783,-3.9836,
17,Hermanus,ZA,73.00,clear sky,-34.4187,19.2345,
18,Saint-Philippe,RE,82.40,light rain,-21.3585,55.7679,
19,Bambous Virieux,MU,80.60,few clouds,-20.3428,57.7575,
23,Jawhar,SO,76.41,clear sky,2.7809,45.5005,


In [62]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # Get the latitude and longitude.
    lat = row["Lat"]
    lng = row["Lng"]
    # 6c. Get latitude and longitude from DataFrame
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.

    try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError): print("Hotel not found... skipping.")


# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [66]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df =hotel_df[hotel_df["Hotel Name"]!= ""]
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,80.89,overcast clouds,3.0707,172.7902,Isles Sunset Lodge
1,Rikitea,PF,77.47,overcast clouds,-23.1203,-134.9692,Pension Maro'i
5,Sao Joao Da Barra,BR,80.60,broken clouds,-21.6403,-41.0511,Pousada Porto De Canoas
11,Mahebourg,MU,80.60,few clouds,-20.4081,57.7000,Shandrani Beachcomber Resort & Spa
13,Sinnamary,GF,80.17,overcast clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
17,Hermanus,ZA,73.00,clear sky,-34.4187,19.2345,Misty Waves Boutique Hotel
18,Saint-Philippe,RE,82.40,light rain,-21.3585,55.7679,"Chambres d'hôte ""La Trinité"""
19,Bambous Virieux,MU,80.60,few clouds,-20.3428,57.7575,Casa Tia Villa
23,Jawhar,SO,76.41,clear sky,2.7809,45.5005,cosoble spare parts
25,Urrao,CO,73.40,broken clouds,6.3152,-76.1342,Finca Hotel Riomanso


In [71]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [72]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """

<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [73]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))